# آليات الانتباه والنماذج المحولة

أحد العيوب الرئيسية للشبكات العصبية المتكررة هو أن جميع الكلمات في التسلسل لها نفس التأثير على النتيجة. يؤدي هذا إلى أداء غير مثالي مع نماذج LSTM القياسية للترميز وفك الترميز في مهام التسلسل إلى التسلسل، مثل التعرف على الكيانات المسماة والترجمة الآلية. في الواقع، غالبًا ما يكون لبعض الكلمات في تسلسل الإدخال تأثير أكبر على المخرجات المتسلسلة مقارنة بغيرها.

لنأخذ نموذج التسلسل إلى التسلسل كمثال، مثل الترجمة الآلية. يتم تنفيذه باستخدام شبكتين متكررتين، حيث تقوم شبكة واحدة (**المشفّر**) بضغط تسلسل الإدخال إلى حالة مخفية، وتقوم الأخرى، **المفكّك**، بفك هذه الحالة المخفية إلى النتيجة المترجمة. المشكلة في هذا النهج هي أن الحالة النهائية للشبكة ستواجه صعوبة في تذكر بداية الجملة، مما يؤدي إلى ضعف جودة النموذج في الجمل الطويلة.

**آليات الانتباه** توفر وسيلة لوزن التأثير السياقي لكل متجه إدخال على كل توقع إخراج للشبكة العصبية المتكررة. يتم تنفيذ ذلك من خلال إنشاء اختصارات بين الحالات الوسيطة لشبكة الإدخال العصبية المتكررة وشبكة الإخراج العصبية المتكررة. بهذه الطريقة، عند توليد رمز الإخراج $y_t$، سنأخذ في الاعتبار جميع الحالات المخفية للإدخال $h_i$، مع معاملات وزن مختلفة $\alpha_{t,i}$.

![صورة توضح نموذج الترميز/فك الترميز مع طبقة انتباه إضافية](../../../../../translated_images/ar/encoder-decoder-attention.7a726296894fb567.webp)
*نموذج الترميز-فك الترميز مع آلية الانتباه الإضافية في [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf)، مقتبس من [هذا المنشور](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

مصفوفة الانتباه $\{\alpha_{i,j}\}$ تمثل الدرجة التي تلعب بها كلمات الإدخال دورًا في توليد كلمة معينة في تسلسل الإخراج. أدناه مثال على مثل هذه المصفوفة:

![صورة توضح محاذاة نموذج RNNsearch-50، مأخوذة من Bahdanau - arviz.org](../../../../../translated_images/ar/bahdanau-fig3.09ba2d37f202a6af.webp)

*الصورة مأخوذة من [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (الشكل 3)*

آليات الانتباه مسؤولة عن الكثير من التقدم الحالي أو القريب من الحالي في معالجة اللغة الطبيعية. ومع ذلك، فإن إضافة الانتباه تزيد بشكل كبير من عدد معلمات النموذج، مما أدى إلى مشكلات في التوسع مع الشبكات العصبية المتكررة. أحد القيود الرئيسية لتوسيع الشبكات العصبية المتكررة هو أن الطبيعة المتكررة للنماذج تجعل من الصعب تجميع وتوازي التدريب. في الشبكات العصبية المتكررة، يجب معالجة كل عنصر من عناصر التسلسل بترتيب متسلسل، مما يعني أنه لا يمكن توازيه بسهولة.

أدى تبني آليات الانتباه مع هذا القيد إلى إنشاء نماذج المحولات التي تمثل الآن أحدث ما توصلت إليه التكنولوجيا، والتي نعرفها ونستخدمها اليوم مثل BERT وOpenGPT3.

## نماذج المحولات

بدلاً من تمرير سياق كل توقع سابق إلى خطوة التقييم التالية، تستخدم **نماذج المحولات** **الترميزات الموضعية** و**الانتباه** لالتقاط سياق الإدخال المعطى ضمن نافذة نصية محددة. الصورة أدناه توضح كيف يمكن للترميزات الموضعية مع الانتباه التقاط السياق ضمن نافذة معينة.

![صورة متحركة توضح كيفية إجراء التقييمات في نماذج المحولات.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

نظرًا لأن كل موضع إدخال يتم تعيينه بشكل مستقل إلى كل موضع إخراج، يمكن للمحولات التوازي بشكل أفضل من الشبكات العصبية المتكررة، مما يتيح نماذج لغوية أكبر وأكثر تعبيرًا. يمكن استخدام كل رأس انتباه لتعلم علاقات مختلفة بين الكلمات، مما يحسن مهام معالجة اللغة الطبيعية.

## بناء نموذج محول بسيط

لا تحتوي مكتبة Keras على طبقة محول مدمجة، ولكن يمكننا بناء واحدة بأنفسنا. كما في السابق، سنركز على تصنيف النصوص باستخدام مجموعة بيانات AG News، ولكن من الجدير بالذكر أن نماذج المحولات تظهر أفضل النتائج في المهام الأكثر صعوبة في معالجة اللغة الطبيعية.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

يجب أن تكون الطبقات الجديدة في Keras من الفئة الفرعية `Layer`، وتنفذ طريقة `call`. لنبدأ بطبقة **التضمين الموضعي**. سنستخدم [بعض التعليمات البرمجية من وثائق Keras الرسمية](https://keras.io/examples/nlp/text_classification_with_transformer/). سنفترض أننا نقوم بملء جميع تسلسلات الإدخال إلى الطول `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

تتكون هذه الطبقة من طبقتين `Embedding`: واحدة لتضمين الرموز (بالطريقة التي ناقشناها سابقًا) والأخرى لتضمين مواضع الرموز. يتم إنشاء مواضع الرموز كسلسلة من الأعداد الطبيعية من 0 إلى `maxlen` باستخدام `tf.range`، ثم يتم تمريرها عبر طبقة التضمين. يتم بعد ذلك جمع متجهي التضمين الناتجين، مما ينتج تمثيلًا مضمنًا موضعيًا للإدخال بالشكل `maxlen`$\times$`embed_dim`.

الآن، دعونا ننفذ كتلة المحول. ستأخذ هذه الكتلة مخرجات طبقة التضمين التي تم تعريفها سابقًا:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

الآن، نحن جاهزون لتعريف نموذج المحول الكامل:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## نماذج BERT Transformer

**BERT** (تمثيلات التشفير ثنائية الاتجاه من المحولات) هو شبكة محولات متعددة الطبقات كبيرة جدًا تحتوي على 12 طبقة في *BERT-base*، و24 طبقة في *BERT-large*. يتم تدريب النموذج مبدئيًا على مجموعة بيانات نصية ضخمة (ويكيبيديا + كتب) باستخدام تدريب غير خاضع للإشراف (التنبؤ بالكلمات المحجوبة في الجملة). خلال مرحلة التدريب المبدئي، يكتسب النموذج مستوى كبيرًا من فهم اللغة، والذي يمكن استغلاله لاحقًا مع مجموعات بيانات أخرى باستخدام التخصيص الدقيق. تُعرف هذه العملية باسم **التعلم بالنقل**.

![صورة من http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/ar/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

هناك العديد من التعديلات على بنية المحولات، بما في ذلك BERT، وDistilBERT، وBigBird، وOpenGPT3، والمزيد، والتي يمكن تخصيصها بدقة.

دعونا نرى كيف يمكننا استخدام نموذج BERT المدرب مسبقًا لحل مشكلة تصنيف التسلسل التقليدية لدينا. سنستعير الفكرة وبعض الشيفرات من [التوثيق الرسمي](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

لتحميل النماذج المدربة مسبقًا، سنستخدم **Tensorflow hub**. أولاً، دعونا نحمل الموجه الخاص بـ BERT:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

من المهم أن تستخدم نفس الـ vectorizer الذي تم تدريب الشبكة الأصلية عليه. بالإضافة إلى ذلك، يقوم BERT vectorizer بإرجاع ثلاثة مكونات:
* `input_word_ids`، وهو تسلسل أرقام الرموز لجملة الإدخال
* `input_mask`، الذي يوضح أي جزء من التسلسل يحتوي على الإدخال الفعلي، وأي جزء هو padding. وهو مشابه للقناع الذي يتم إنتاجه بواسطة طبقة `Masking`
* `input_type_ids` يُستخدم لمهام نمذجة اللغة، ويسمح بتحديد جملتين إدخاليتين في تسلسل واحد.

بعد ذلك، يمكننا إنشاء مستخرج ميزات BERT:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


لذا، تقوم طبقة BERT بإرجاع عدد من النتائج المفيدة:
* `pooled_output` هو نتيجة متوسط جميع الرموز في التسلسل. يمكنك اعتباره تمثيلاً ذكياً للمعنى العام للشبكة بأكملها. وهو يعادل ناتج الطبقة `GlobalAveragePooling1D` في نموذجنا السابق.
* `sequence_output` هو ناتج الطبقة الأخيرة من المحول (يتوافق مع ناتج `TransformerBlock` في نموذجنا أعلاه).
* `encoder_outputs` هي نواتج جميع طبقات المحول. بما أننا قمنا بتحميل نموذج BERT مكون من 4 طبقات (كما يمكنك أن تخمن من الاسم الذي يحتوي على `4_H`)، فإنه يحتوي على 4 موترات. الأخير منها هو نفسه `sequence_output`.

الآن سنقوم بتعريف نموذج التصنيف الشامل. سنستخدم *تعريف النموذج الوظيفي*، حيث نقوم بتحديد مدخلات النموذج، ثم نقدم سلسلة من التعبيرات لحساب مخرجاته. سنجعل أيضاً أوزان نموذج BERT غير قابلة للتدريب، وسنقوم بتدريب المصنف النهائي فقط:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


على الرغم من أن هناك عددًا قليلاً من المعاملات القابلة للتدريب، إلا أن العملية بطيئة جدًا، لأن مستخرج ميزات BERT يتطلب حسابات مكثفة. يبدو أننا لم نتمكن من تحقيق دقة معقولة، إما بسبب نقص التدريب أو نقص في معاملات النموذج.

دعونا نحاول فك تجميد أوزان BERT وتدريبه أيضًا. يتطلب هذا معدل تعلم صغير جدًا، بالإضافة إلى استراتيجية تدريب أكثر حذرًا مع **الإحماء**، باستخدام مُحسّن **AdamW**. سنستخدم حزمة `tf-models-official` لإنشاء المُحسّن:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

كما ترى، التدريب يسير ببطء شديد - ولكن قد ترغب في التجربة وتدريب النموذج لعدة دورات (5-10) لترى ما إذا كان بإمكانك الحصول على أفضل نتيجة مقارنة بالنهج التي استخدمناها سابقًا.

## مكتبة Huggingface Transformers

طريقة أخرى شائعة جدًا (وأبسط قليلاً) لاستخدام نماذج Transformer هي [حزمة HuggingFace](https://github.com/huggingface/)، التي توفر مكونات بسيطة لمهام معالجة اللغة الطبيعية المختلفة. وهي متاحة لكل من Tensorflow وPyTorch، وهو إطار عمل آخر شائع للشبكات العصبية.

> **ملاحظة**: إذا لم تكن مهتمًا برؤية كيفية عمل مكتبة Transformers - يمكنك تخطي نهاية هذا الدفتر، لأنك لن ترى أي شيء مختلف بشكل جوهري عما قمنا به أعلاه. سنقوم بتكرار نفس خطوات تدريب نموذج BERT باستخدام مكتبة مختلفة ونموذج أكبر بشكل كبير. وبالتالي، تتضمن العملية بعض التدريب الطويل جدًا، لذا قد ترغب فقط في استعراض الكود.

دعونا نرى كيف يمكن حل مشكلتنا باستخدام [Huggingface Transformers](http://huggingface.co).


أول شيء نحتاج إلى القيام به هو اختيار النموذج الذي سنستخدمه. بالإضافة إلى بعض النماذج المدمجة، يحتوي Huggingface على [مستودع نماذج عبر الإنترنت](https://huggingface.co/models)، حيث يمكنك العثور على العديد من النماذج المدربة مسبقًا من قبل المجتمع. يمكن تحميل واستخدام جميع هذه النماذج فقط من خلال توفير اسم النموذج. سيتم تنزيل جميع الملفات الثنائية المطلوبة للنموذج تلقائيًا.

في بعض الأحيان قد تحتاج إلى تحميل نماذجك الخاصة، وفي هذه الحالة يمكنك تحديد الدليل الذي يحتوي على جميع الملفات ذات الصلة، بما في ذلك معلمات الـ tokenizer، ملف `config.json` الذي يحتوي على معلمات النموذج، الأوزان الثنائية، وما إلى ذلك.

من اسم النموذج، يمكننا إنشاء كل من النموذج والـ tokenizer. لنبدأ مع الـ tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

يحتوي كائن `tokenizer` على وظيفة `encode` التي يمكن استخدامها مباشرة لترميز النص:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

يمكننا أيضًا استخدام أداة تقسيم النص لترميز تسلسل بطريقة مناسبة لتمريره إلى النموذج، أي تضمين الحقول مثل `token_ids` و `input_mask`، وما إلى ذلك. يمكننا أيضًا تحديد أننا نريد متغيرات Tensorflow عن طريق تقديم الوسيطة `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

في حالتنا، سنستخدم نموذج BERT المدرب مسبقًا والذي يُسمى `bert-base-uncased`. يشير *Uncased* إلى أن النموذج غير حساس لحالة الأحرف.

عند تدريب النموذج، نحتاج إلى تقديم تسلسل مرمز كمدخل، وبالتالي سنقوم بتصميم خط معالجة البيانات. وبما أن `tokenizer.encode` هي وظيفة في بايثون، سنستخدم نفس النهج كما في الوحدة السابقة من خلال استدعائها باستخدام `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

الآن يمكننا تحميل النموذج الفعلي باستخدام حزمة `BertForSequenceClassification`. يضمن ذلك أن النموذج الخاص بنا يحتوي بالفعل على البنية المطلوبة للتصنيف، بما في ذلك المصنف النهائي. ستظهر رسالة تحذير تشير إلى أن أوزان المصنف النهائي غير مهيأة، وأن النموذج سيحتاج إلى تدريب مسبق - وهذا أمر طبيعي تمامًا، لأنه بالضبط ما نحن على وشك القيام به!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


كما ترى من `summary()`، يحتوي النموذج على ما يقرب من 110 مليون معلمة! من المفترض، إذا أردنا مهمة تصنيف بسيطة على مجموعة بيانات صغيرة نسبيًا، فلا نريد تدريب طبقة BERT الأساسية:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


الآن نحن جاهزون لبدء التدريب!

> **ملاحظة**: تدريب نموذج BERT كامل النطاق يمكن أن يكون مستهلكًا جدًا للوقت! لذلك سنقوم فقط بتدريبه لأول 32 دفعة. هذا فقط لإظهار كيفية إعداد تدريب النموذج. إذا كنت مهتمًا بتجربة التدريب كامل النطاق - فقط قم بإزالة معلمات `steps_per_epoch` و `validation_steps`، واستعد للانتظار!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


إذا قمت بزيادة عدد التكرارات وانتظرت لفترة كافية، وقمت بالتدريب لعدة دورات، يمكنك أن تتوقع أن تصنيف BERT سيعطينا أفضل دقة! ذلك لأن BERT يفهم بالفعل هيكل اللغة بشكل جيد، وكل ما نحتاجه هو تحسين المصنف النهائي. ومع ذلك، نظرًا لأن BERT نموذج كبير، فإن عملية التدريب بأكملها تستغرق وقتًا طويلًا، وتتطلب قوة حسابية كبيرة! (وحدة معالجة الرسومات GPU، ويفضل أن تكون أكثر من واحدة).

> **ملاحظة:** في مثالنا، كنا نستخدم أحد أصغر نماذج BERT المدربة مسبقًا. هناك نماذج أكبر من المحتمل أن تقدم نتائج أفضل.


## النقاط الرئيسية

في هذه الوحدة، استعرضنا أحدث بنى النماذج المعتمدة على **المحولات**. قمنا بتطبيقها على مهمة تصنيف النصوص، وبالمثل، يمكن استخدام نماذج BERT لاستخراج الكيانات، والإجابة على الأسئلة، وغيرها من مهام معالجة اللغة الطبيعية.

تمثل نماذج المحولات أحدث ما توصلت إليه التكنولوجيا في مجال معالجة اللغة الطبيعية، وفي معظم الحالات، ينبغي أن تكون الحل الأول الذي تبدأ بتجربته عند تنفيذ حلول مخصصة لمعالجة اللغة الطبيعية. ومع ذلك، فإن فهم المبادئ الأساسية للشبكات العصبية التكرارية التي تمت مناقشتها في هذه الوحدة يُعد أمرًا بالغ الأهمية إذا كنت ترغب في بناء نماذج عصبية متقدمة.



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة الآلية [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية هو المصدر الموثوق. للحصول على معلومات حساسة أو هامة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
